In [127]:
import pandas as pd
import numpy as np
import datetime as dt
import math as mt

In [128]:
peak_df = pd.DataFrame(columns=['date','peak (kW)'])
household_kWh_df = pd.DataFrame(columns=['name', 'usage (kWh)'])
now_month = 2

# Set Peak Data
year = 2020
months = [3,4,5,6,7,8,9,10,11,12,1,2]
peak_kW = [95,95,94,90,91,72,66,87,91,85,100,95]
household_kWh = [150, 180, 220, 210, 310, 300, 270, 190, 250, 260]

for idx,month in enumerate(months):
    if month == 1:
        year += 1
    peak_df = peak_df.append({
        "date": dt.datetime(
            year,
            month,
            1
        ),
        "peak (kW)": peak_kW[idx]
    }, ignore_index=True)
    
# Set Household Data
for _ in range(0, 10):
    household_kWh_df = household_kWh_df.append({
        "name": str(_ + 1) + "01호",
        "usage (kWh)": household_kWh[_]
    }, ignore_index=True)
    
peak_df
household_kWh_df

,name,usage (kWh)
0,101호,150
1,201호,180
2,301호,220
3,401호,210
4,501호,310
5,601호,300
6,701호,270
7,801호,190
8,901호,250
9,1001호,260


In [158]:
# 1~6, 9~12
low_pressure_fee = [
    [0,200,910,88.3],
    [201,400,1600, 182.9],
    [401,float('inf'),7300, 275.6]
]
# 7~8
low_pressure_fee_summer = [
    [0,300,910, 88.3],
    [301,450,1600, 182.9],
    [451,float('inf'),7300, 275.6]
]

general_fee = {
    "저압": [6160, 100.7, 60.2, 87.3],
    "고압 A": [
        [7170, 110.9, 66.9, 98.6],
        [8230, 106.9, 62.6, 93.3]
    ],
    "고압 B": [
        [7170, 108.8, 65.8, 95.6],
        [8230, 103.5, 60.5, 90.3]
    ]
}
env_fee = 5.3
fuel_fee = 3
VAT = 0.1
FUND = 0.037

class HouseHold:
    def __init__(self,
                name, kwh, contract):
        self.name = name
        self.kwh = kwh
        self.contract = contract
        
    @property
    def basic(self):
        fee = None
        for _ in self.contract:
            if _[0] <= self.kwh and _[1] >= self.kwh:
                fee = _[2]
                break
        return fee
    
    @property
    def elec_rate(self):
        kwh = self.kwh
        fee = 0
        for _ in self.contract:
            if kwh <= _[1]:
                fee += (kwh * _[3])
                break
            else:
                kwh -= _[1]
                fee += (_[1] * _[3])
        
        return mt.floor(fee)
    
    @property
    def env(self):
        return mt.floor(self.kwh * env_fee)
    
    @property
    def fuel(self):
        return mt.floor(self.kwh * fuel_fee)
        
    @property
    def guarantee(self):
        if self.kwh <= 200:
            return 4000
        else:
            return 0
        
    @property
    def total(self):
        return self.basic + self.elec_rate\
            - self.guarantee + self.env - self.fuel
    
    @property
    def vat(self):
        return round(self.total * 0.1)
    
    @property
    def fund(self):
        return mt.floor(self.total * 0.037 * 0.1) * 10
    
    @property
    def total_last(self):
        return mt.floor((self.total + self.vat + self.fund) * 0.1) * 10
        
                

class ManagementOffice:
    def __init__(self, 
                 month, peaks, households, APT, # datas
                 contract,
                 general_fee_info=None):
        print("[관리사무소] Init.")
        
        self.peaks = peaks
        self.APT = APT
        
        self.contract = contract
        self.select_fee(month, contract, general_fee_info)
        
        print("[관리사무소] 가구 객체화")
        self.households = list()
        for idx in households.index:
            self.households.append(
                HouseHold(
                    name=households.iloc[idx]['name'],
                    kwh=households.iloc[idx]['usage (kWh)'],
                    contract=self.fee[0]
                )
            )

    # 요금제 셋팅 메서드
    def select_fee(self, month, contract, general_fee_info):
            print("[관리사무소] 계약 정보 셋팅")

            if contract == "종합계약":
                household_fee = None
                common_fee = None
                if (month >= 1 and month <= 6) or\
                    (month >= 9 and month <= 12):
                    household_fee = low_pressure_fee
                else:
                    household_fee = low_pressure_fee_summer

                if general_fee_info == None:
                    raise Exception("종합계약은 일반용 전력 정보를 포함해야 합니다.\n")

                tmp_general_fee = None
                if general_fee_info == "저압":
                    tmp_general_fee = general_fee[general_fee_info]
                elif len(general_fee_info) == 2:
                    tmp_general_fee = general_fee[general_fee_info[0]]
                    tmp_general_fee = tmp_general_fee[general_fee_info[1]]
                else: 
                    raise Exception("일반용 전력 설정이 올바르지 않습니다\n"\
                                + "저압, [고압 A, 0 or 1], [고압 B, 0 or 1]")

                if month >= 6 and month <= 8:
                    common_fee = [tmp_general_fee[0], tmp_general_fee[1]]
                elif (month >= 3 and month <= 5) or\
                      (month >= 9 and month <= 10):
                    common_fee = [tmp_general_fee[0], tmp_general_fee[2]]
                elif (month >= 1 and month <= 2) or\
                      (month >= 11 and month <= 12):
                    common_fee = [tmp_general_fee[0], tmp_general_fee[3]]

                self.fee = [household_fee, common_fee]

            print("[관리사무소] 계약 정보 셋팅 완료\n")
            print(self.fee)        
            print("")

In [164]:
calc = ManagementOffice(
    month=2,
    peaks=peak_df, 
    households=household_kWh_df,
    APT=3000,
    contract="종합계약",
    general_fee_info=['고압 A', 1]
)

idx = 5
print("기본요금",calc.households[idx].basic)
print("전력량 요금",calc.households[idx].elec_rate)
print("기후환경요금",calc.households[idx].env)
print("연료비 조정액",calc.households[idx].fuel)
print("전기요금계",calc.households[idx].total)
print("부가가치세",calc.households[idx].vat)
print("전력산업기반기금",calc.households[idx].fund)
print("부가가치세, 전력산업기반기금 적용",calc.households[idx].total_last)



[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[[0, 200, 910, 88.3], [201, 400, 1600, 182.9], [401, inf, 7300, 275.6]], [8230, 93.3]]

[관리사무소] 가구 객체화
기본요금 1600
전력량 요금 35950
기후환경요금 1590
연료비 조정액 900
전기요금계 38240
부가가치세 3824
전력산업기반기금 1410
부가가치세, 전력산업기반기금 적용 43470
